In [2]:
import pandas as pd
import backtesting
# from backtesting import Backtest, Strategy
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import *
import numpy as np
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

import ipywidgets as widgets

C:\Users\Karolina\anaconda3\envs\Magisterka\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
data = pd.read_csv('data/decisions_EURUSD_5Y.csv')

In [4]:
data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Name,Return,Up,...,MACD_decision,diff_MACD,diff_MACD_prev,fuzz_MACD_strategy,dec_bollinger,diff_cl_BU,diff_cl_BL,prev_diff_cl_BU,prev_diff_cl_BL,fuzz_BB_strategy
0,2019-05-02,1.120185,1.121818,1.117356,1.120084,1.120084,0,EURUSD=X,-0.001356,0.000000,...,0,0.000048,0.000000,0,0,-0.013554,0.006675,-0.012144,0.007844,0
1,2019-05-03,1.117256,1.120072,1.114132,1.117356,1.117356,0,EURUSD=X,-0.002439,0.000000,...,-1,-0.000081,0.000048,0,0,-0.016387,0.004574,-0.013554,0.006675,0
2,2019-05-06,1.118706,1.120448,1.117993,1.118581,1.118581,0,EURUSD=X,0.001096,0.001096,...,0,-0.000063,-0.000081,0,0,-0.015189,0.006163,-0.016387,0.004574,0
3,2019-05-07,1.120448,1.121957,1.117606,1.120185,1.120185,0,EURUSD=X,0.001433,0.001433,...,1,0.000072,-0.000063,0,0,-0.013316,0.008045,-0.015189,0.006163,0
4,2019-05-08,1.118956,1.121500,1.118956,1.119107,1.119107,0,EURUSD=X,-0.000963,0.000000,...,0,0.000100,0.000072,0,0,-0.013974,0.007306,-0.013316,0.008045,0
5,2019-05-09,1.119746,1.125100,1.117568,1.119833,1.119833,0,EURUSD=X,0.000649,0.000649,...,0,0.000176,0.000100,0,0,-0.012623,0.008189,-0.013974,0.007306,0
6,2019-05-10,1.121856,1.125239,1.121856,1.122208,1.122208,0,EURUSD=X,0.002119,0.002119,...,0,0.000382,0.000176,0,0,-0.009882,0.010585,-0.012623,0.008189,0
7,2019-05-13,1.123671,1.126342,1.122410,1.123646,1.123646,0,EURUSD=X,0.001280,0.001280,...,0,0.000599,0.000382,0,0,-0.007310,0.011574,-0.009882,0.010585,0
8,2019-05-14,1.123242,1.124606,1.120486,1.123255,1.123255,0,EURUSD=X,-0.000348,0.000000,...,0,0.000692,0.000599,0,0,-0.006398,0.010604,-0.007310,0.011574,0
9,2019-05-15,1.120360,1.122272,1.118043,1.120448,1.120448,0,EURUSD=X,-0.002502,0.000000,...,0,0.000548,0.000692,0,0,-0.008067,0.007465,-0.006398,0.010604,0


### Sprawdzanie sygnałów w kolumnach ostrych i rozmytych

In [5]:
# Zliczenie przypadków, w których wartości są przeciwne

rsi_count = ((data['dec_rsi'] == 1) & (data['Signal_RSI_fuzz_system'] == -1) | (data['dec_rsi'] == -1) & (data['Signal_RSI_fuzz_system'] == 1)).sum()
print("rsi_count", rsi_count)

williams_count = ((data['dec_will'] == 1) & (data['Signal_william_fuzz_system'] == -1) | (data['dec_will'] == -1) & (data['Signal_william_fuzz_system'] == 1)).sum()
print("williams_count", williams_count)

# cross_MA - ostre decyzje
ma_count = ((data['cross_MA'] == 1) & (data['fuzz_MA_strategy'] == -1) | (data['cross_MA'] == -1) & (data['fuzz_MA_strategy'] == 1)).sum()
print("ma_count", ma_count)

macd_count = ((data['MACD_decision'] == 1) & (data['fuzz_MACD_strategy'] == -1) | (data['MACD_decision'] == -1) & (data['fuzz_MACD_strategy'] == 1)).sum()
print("macd_count", macd_count)


bb_count = ((data['dec_bollinger'] == 1) & (data['fuzz_BB_strategy'] == -1) | (data['dec_bollinger'] == -1) & (data['fuzz_BB_strategy'] == 1)).sum()

print("bb_count", bb_count)


rsi_count 0
williams_count 0
ma_count 1
macd_count 0
bb_count 0


In [6]:
opposite_rows = data[((data['cross_MA'] == 1) & (data['fuzz_MA_strategy'] == -1)) | ((data['cross_MA'] == -1) & (data['fuzz_MA_strategy'] == 1))]
opposite_rows[['Date', '5MA', '15MA', 'cross_MA', 'fuzz_MA_strategy', 'diff_MA', 'diff_MA_prev']]


,Date,5MA,15MA,cross_MA,fuzz_MA_strategy,diff_MA,diff_MA_prev
251,2020-04-20,1.090778,1.090844,-1,1,-0.000066,-0.000566


<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExNWcyanR3eWUxYTRsNHpyZTAxbmoyNHcxOTJnbWtqcnl5NHRyN2hqMCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/jPAdK8Nfzzwt2/giphy.gif" width="350" align="center">

Na ten moment nie wiem czemu tutaj jest rozbieżność

In [7]:
data = data.set_index(pd.DatetimeIndex(data['Date']))

## Tylko sygnały

### Ostre

In [8]:
class CustomStrategy(SignalStrategy):
    def init(self):
        super().init()
        
        self.set_signal(data['dec_rsi'])
      
        
bt = Backtest(data, CustomStrategy, cash=10000, 
              commission=0.002, trade_on_close=False, hedging = False, exclusive_orders = True)

stats = bt.run()
bt.plot()
stats

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   96.028037
Equity Final [$]                   9999.21572
Equity Peak [$]                       10000.0
Return [%]                          -0.007843
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -0.001539
Volatility (Ann.) [%]                  0.0008
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.007968
Avg. Drawdown [%]                   -0.007968
Max. Drawdown Duration     1727 days 00:00:00
Avg. Drawdown Duration     1727 days 00:00:00
# Trades                                  304
Win Rate [%]                        34.868421
Best Trade [%]                       3.345059
Worst Trade [%]                     -4.160291
Avg. Trade [%]                    

### Rozmyte

In [19]:
class CustomStrategy(SignalStrategy):
    def init(self):
        super().init()
        
        self.set_signal(data['Signal_RSI_fuzz_system'])
      
        
bt = Backtest(data, CustomStrategy, cash=10000, 
              commission=0.002, trade_on_close=False, hedging = False, exclusive_orders = True)

stats = bt.run()
bt.plot()
stats

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   97.819315
Equity Final [$]                  9999.212488
Equity Peak [$]                  10000.008594
Return [%]                          -0.007875
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -0.001546
Volatility (Ann.) [%]                0.000806
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.008065
Avg. Drawdown [%]                   -0.002698
Max. Drawdown Duration     1750 days 00:00:00
Avg. Drawdown Duration      585 days 00:00:00
# Trades                                  386
Win Rate [%]                        35.751295
Best Trade [%]                       2.784302
Worst Trade [%]                     -4.588674
Avg. Trade [%]                    

## Strategia z tp i sl

### Ostre

In [18]:
class CustomStrategy(SignalStrategy):
    
    price_upper = 0.1  # różnica w górę
    price_lower = 0.02  # różnica w dół
    
    def init(self):
        super().init()
        self.rsi = self.I(lambda x: data['dec_rsi'], 'rsi')
        self.last_trade_date = None

    def next(self):
        close = self.data.Close[-1]
        signal = self.rsi[-1] 

        upper = close * (1 + self.price_upper)  # upper jako price_upper% różnica w górę
        lower = close * (1 - self.price_lower)  # lower jako price_lower% różnica w dół   
        
        size_transaction = .7
        
        if signal == -1:
            if self.position.is_long:
                self.sell(size=size_transaction, sl = upper, tp=lower)
            elif self.position.is_short:
                pass
            else:
                self.sell(size=size_transaction, sl = upper, tp=lower)
        elif signal == 1:
            if self.position.is_long:
                pass
            elif self.position.is_short:
                self.buy(size=size_transaction, sl=lower, tp = upper)

            else:
                self.buy(size=size_transaction,sl=lower, tp = upper)



bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False, exclusive_orders=True)
stats = bt.run()
bt.plot()
stats


Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   92.834891
Equity Final [$]                  9327.414247
Equity Peak [$]                  10449.166947
Return [%]                          -6.725858
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -1.357222
Volatility (Ann.) [%]                4.893971
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -12.201869
Avg. Drawdown [%]                   -2.745491
Max. Drawdown Duration     1465 days 00:00:00
Avg. Drawdown Duration      250 days 00:00:00
# Trades                                   43
Win Rate [%]                        53.488372
Best Trade [%]                       3.131685
Worst Trade [%]                     -9.868757
Avg. Trade [%]                    

### Rozmyte

In [21]:
class CustomStrategy(SignalStrategy):
    
    price_upper = 0.1  # różnica w górę
    price_lower = 0.02  # różnica w dół
    
    def init(self):
        super().init()
        self.rsi = self.I(lambda x: data['Signal_RSI_fuzz_system'], 'rsi')
        self.last_trade_date = None

    def next(self):
        close = self.data.Close[-1]
        signal = self.rsi[-1] 

        upper = close * (1 + self.price_upper)  # upper jako price_upper% różnica w górę
        lower = close * (1 - self.price_lower)  # lower jako price_lower% różnica w dół   
        
        size_transaction = .7
        
        if signal == -1:
            if self.position.is_long:
                self.sell(size=size_transaction, sl = upper, tp=lower)
            elif self.position.is_short:
                pass
            else:
                self.sell(size=size_transaction, sl = upper, tp=lower)
        elif signal == 1:
            if self.position.is_long:
                pass
            elif self.position.is_short:
                self.buy(size=size_transaction, sl=lower, tp = upper)

            else:
                self.buy(size=size_transaction,sl=lower, tp = upper)



bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False, exclusive_orders=True)
stats = bt.run()
bt.plot()
stats


Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   94.548287
Equity Final [$]                 10101.282564
Equity Peak [$]                  10481.764696
Return [%]                           1.012826
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    0.197975
Volatility (Ann.) [%]                5.002236
Sharpe Ratio                         0.039577
Sortino Ratio                        0.056238
Calmar Ratio                         0.026505
Max. Drawdown [%]                   -7.469197
Avg. Drawdown [%]                    -2.15237
Max. Drawdown Duration     1465 days 00:00:00
Avg. Drawdown Duration      291 days 00:00:00
# Trades                                   51
Win Rate [%]                        58.823529
Best Trade [%]                       3.372285
Worst Trade [%]                     -6.189283
Avg. Trade [%]                    

***
***

## Pozostałe próby 

In [9]:
class CustomStrategy(SignalStrategy):
    
    price_delta = 0.01 # różnica w cenie
    
    def init(self):
        super().init()
        self.rsi = self.I(lambda x: data['dec_rsi'], 'rsi')

    def next(self):
        close = self.data.Close[-1]
        signal = self.rsi[-1] 

        upper, lower = close * (1 + np.r_[1, -1] * self.price_delta)  # price_delta * 100% różnicy pomiędzy ceną zamknięcia 
        
        size_transaction = .7
        
        if not self.position:
            if signal == 1 and self._broker._cash == 10000: 
                self.buy(size=size_transaction, tp = upper)
        elif self.position:    
            if signal == 1 and not self.position.is_long:
                self.buy(size=size_transaction, tp = upper)
            elif signal == -1:
                self.sell(size=size_transaction, sl = upper)
                
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False, exclusive_orders=True)
stats = bt.run()
bt.plot()
stats


Start                     2019-05-01 00:00:00
End                       2024-04-03 00:00:00
Duration                   1799 days 00:00:00
Exposure Time [%]                     8.48249
Equity Final [$]                   9849.75399
Equity Peak [$]                       10000.0
Return [%]                           -1.50246
Buy & Hold Return [%]               -3.989744
Return (Ann.) [%]                   -0.296442
Volatility (Ann.) [%]                0.975841
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.796206
Avg. Drawdown [%]                   -2.796206
Max. Drawdown Duration     1727 days 00:00:00
Avg. Drawdown Duration     1727 days 00:00:00
# Trades                                   14
Win Rate [%]                        35.714286
Best Trade [%]                       0.800389
Worst Trade [%]                     -1.480001
Avg. Trade [%]                    

In [28]:
class CustomStrategy(SignalStrategy):
    
    price_delta = 0.1 # różnica w cenie
    #min_days_between_trades = 10  # minimalna liczba dni pomiędzy transakcjami
    
    def init(self):
        super().init()
        self.rsi = self.I(lambda x: data['dec_rsi'], 'rsi')
        self.last_trade_date = None

    def next(self):
        close = self.data.Close[-1]
        signal = self.rsi[-1] 

        upper, lower = close * (1 + np.r_[1, -1] * self.price_delta)  # price_delta * 100% różnicy pomiędzy ceną zamknięcia 
        
        size_transaction = .7
        
        if self.last_trade_date is not None:
            days_since_last_trade = (self.data.index[-1] - self.last_trade_date).days
        else:
            days_since_last_trade = None  # zmienione na None, aby umożliwić pierwszą transakcję
        
        #if not self.position:
           # if signal == 1 and self._broker._cash == 10000 :#and days_since_last_trade == None: 
            #    self.buy(size=size_transaction, tp = upper)
             #   self.last_trade_date = self.data.index[-1]
        if signal == -1:
            if self.position.is_long:
                self.sell(size=size_transaction, sl = upper, tp=lower)
            elif self.position.is_short:
                pass
            else:
                self.sell(size=size_transaction, sl = upper, tp=lower)
            #   self.last_trade_date = self.data.index[-1]
        elif signal == 1:
            if self.position.is_long:
                pass
            elif self.position.is_short:
                self.buy(size=size_transaction, sl=lower, tp = upper)

            else:
                self.buy(size=size_transaction,sl=lower, tp = upper)
             #   self.last_trade_date = self.data.index[-1]
                
#         elif self.position:    
#             if signal == 1 and not self.position.is_long and days_since_last_trade >= self.min_days_between_trades:
#                 self.buy(size=size_transaction, tp = upper)
#                 self.last_trade_date = self.data.index[-1]

#             elif signal == -1 and days_since_last_trade >= self.min_days_between_trades:
#                 self.sell(size=size_transaction, sl = upper)
#                 self.last_trade_date = self.data.index[-1]



bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False, exclusive_orders=True)
stats = bt.run()
bt.plot()
stats


Start                     2019-05-01 00:00:00
End                       2024-04-03 00:00:00
Duration                   1799 days 00:00:00
Exposure Time [%]                    97.66537
Equity Final [$]                  8896.584012
Equity Peak [$]                   10533.82908
Return [%]                          -11.03416
Buy & Hold Return [%]               -3.989744
Return (Ann.) [%]                   -2.266775
Volatility (Ann.) [%]                5.061134
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -16.962737
Avg. Drawdown [%]                   -3.745175
Max. Drawdown Duration     1465 days 00:00:00
Avg. Drawdown Duration      292 days 00:00:00
# Trades                                   29
Win Rate [%]                        55.172414
Best Trade [%]                       3.498213
Worst Trade [%]                     -9.868757
Avg. Trade [%]                    

In [10]:
class CustomStrategy(SignalStrategy):
    
    price_delta = 0.018 # różnica w cenie
    min_days_between_trades = 20  # minimalna liczba dni pomiędzy transakcjami
    max_market_volatility = 0.08  # maksymalna dopuszczalna zmienność rynku
    vol_market_days = 20
    
    def init(self):
        super().init()
        self.rsi = self.I(lambda x: data['dec_rsi'], 'rsi')
        self.last_trade_date = None

    def next(self):
        close = self.data.Close[-1]
        signal = self.rsi[-1] 

        upper, lower = close * (1 + np.r_[1, -1] * self.price_delta)  # price_delta * 100% różnicy pomiędzy ceną zamknięcia 
        
        size_transaction = .7
        
        if self.last_trade_date is not None:
            days_since_last_trade = (self.data.index[-1] - self.last_trade_date).days
        else:
            days_since_last_trade = None  # zmienione na None, aby umożliwić pierwszą transakcję

                
        # konwertowanie numpy do panas
        returns = pd.Series(self.data['Return'])

        # zmienność rynkowa jako odchylenie standardowe zwrotów z ostatnich 10 dni
        market_volatility = returns.rolling(self.vol_market_days).std().iloc[-1]
        
       
        if not self.position and market_volatility <= self.max_market_volatility:
            if signal == 1 and self._broker._cash == 10000 and days_since_last_trade == None: 
                self.buy(size=size_transaction, tp = upper)
                self.last_trade_date = self.data.index[-1]
        elif self.position and market_volatility <= self.max_market_volatility:    
            if signal == 1 and not self.position.is_long and days_since_last_trade >= self.min_days_between_trades:
                self.buy(size=size_transaction, tp = upper)
                self.last_trade_date = self.data.index[-1]

            elif signal == -1 and days_since_last_trade >= self.min_days_between_trades:
                self.sell(size=size_transaction, sl = upper)
                self.last_trade_date = self.data.index[-1]


bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False, exclusive_orders=True)
stats = bt.run()
bt.plot()
stats

Start                     2019-05-01 00:00:00
End                       2024-04-03 00:00:00
Duration                   1799 days 00:00:00
Exposure Time [%]                   13.229572
Equity Final [$]                  9933.103281
Equity Peak [$]                  10062.769224
Return [%]                          -0.668967
Buy & Hold Return [%]               -3.989744
Return (Ann.) [%]                   -0.131545
Volatility (Ann.) [%]                1.366333
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.445849
Avg. Drawdown [%]                    -2.00679
Max. Drawdown Duration     1490 days 00:00:00
Avg. Drawdown Duration      576 days 00:00:00
# Trades                                    6
Win Rate [%]                        66.666667
Best Trade [%]                       0.739518
Worst Trade [%]                     -1.700505
Avg. Trade [%]                    